# 케라스와 텐서플로 허브를 사용한 영화 리뷰 텍스트 분류하기

이 노트북은 리뷰의 텍스트를 사용하여 영화 리뷰를 *긍정적* 또는 *부정적*으로 분류합니다. *이진(* 또는 2-클래스 분류인 이 예는 광범위하게 적용할 수 있는 중요한 머신러닝 응용 사례입니다.

이 튜토리얼은 [TensorFlow Hub](https://tfhub.dev) 및 Keras를 사용한 전이 학습의 기본적인 응용을 보여줍니다.

여기서 사용하는 [IMDB 데이터세트](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)에는 [인터넷 영화 데이터베이스](https://www.imdb.com/)에서 가져온 50,000개의 영화 리뷰 텍스트가 포함되어 있습니다. 훈련용 리뷰 25,000개와 테스트용 리뷰 25,000개로 나뉩니다. 훈련 및 테스트 세트는 *균형을 이룹니다*. 즉, 동일한 수의 긍정적인 리뷰와 부정적인 리뷰가 포함되어 있습니다.

이 노트북은 높은 수준의 API인 [`tf.keras`](https://www.tensorflow.org/guide/keras)를 사용하여 TensorFlow에서 모델을 빌드 및 훈련하고, 단일 코드 줄로 [TFHub](https://tfhub.dev)로부터 훈련된 모델을 로드하기 위한 라이브러리인 [`tensorflow_hub`](https://www.tensorflow.org/hub)를 사용합니다. `tf.keras`를 사용한 고급 텍스트 분류 튜토리얼에 대해서는 [MLCC 텍스트 분류 가이드](https://developers.google.com/machine-learning/guides/text-classification/)를 참조하세요.

In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python -m pip install --upgrade pip' command.


In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.6.0
Eager mode:  True
Hub version:  0.12.0


GPU is available


## IMDB 데이터셋 다운로드

IMDB 데이터셋은 [imdb reviews](https://www.tensorflow.org/datasets/catalog/imdb_reviews) 또는 [텐서플로 데이터셋](https://www.tensorflow.org/datasets)(TensorFlow datasets)에 포함되어 있습니다. 다음 코드는 IMDB 데이터셋을 컴퓨터(또는 코랩 런타임)에 다운로드합니다:

In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## 데이터 탐색

잠시 데이터 형태를 알아 보죠. 이 데이터셋의 샘플은 전처리된 정수 배열입니다. 이 정수는 영화 리뷰에 나오는 단어를 나타냅니다. 레이블(label)은 정수 0 또는 1입니다. 0은 부정적인 리뷰이고 1은 긍정적인 리뷰입니다.

처음 10개의 샘플을 출력해 보겠습니다.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

처음 10개의 레이블도 출력해 보겠습니다.

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 모델 구성

신경망은 층을 쌓아서 만듭니다. 여기에는 세 개의 중요한 구조적 결정이 필요합니다:

- 어떻게 텍스트를 표현할 것인가?
- 모델에서 얼마나 많은 층을 사용할 것인가?
- 각 층에서 얼마나 많은 *은닉 유닛*(hidden unit)을 사용할 것인가?

이 예제의 입력 데이터는 문장으로 구성됩니다. 예측할 레이블은 0 또는 1입니다.

텍스트를 표현하는 한 가지 방법은 문장을 임베딩 벡터로 변환하는 것입니다. 사전 훈련 된 텍스트 임베딩을 첫 번째 레이어로 사용할 수 있으며, 두 가지 이점이 있습니다.

- 텍스트 전처리에 대해 걱정할 필요가 없습니다.
- 전이 학습에 따른 이점이 있습니다.
- 임베딩은 고정 크기이기 때문에 처리 과정이 단순해집니다.

이 예에서는 [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)라고 하는 [TensorFlow Hub](https://tfhub.dev)에서 **사전 훈련된 텍스트 임베딩 모델**을 사용합니다.

이 튜토리얼에서 사용할 수 있는 TFHub의 다른 많은 사전 훈련된 텍스트 임베딩이 있습니다.

- [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) - [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)와 동일한 데이터에 동일한 NNLM 아키텍처로 훈련하지만 임베딩 차원이 더 큽니다. 보다 큰 차원의 임베딩은 작업을 개선할 수 있지만 모델을 훈련하는 데 더 오래 걸릴 수 있습니다.
- [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) - [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2)와 동일하지만 구두점 제거와 같은 추가적인 텍스트 정규화가 있습니다. 이는 작업의 텍스트에 추가 문자나 구두점이 포함된 경우 도움이 될 수 있습니다.
- [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) - DAN(deep averaging network) 인코더로 훈련된 512 차원 임베딩을 생성하는 훨씬 더 큰 모델입니다.

그 밖에도 많이 있습니다! TFHub에서 더 많은 [텍스트 임베딩 모델](https://tfhub.dev/s?module-type=text-embedding)을 찾아보세요.

먼저 문장을 임베딩시키기 위해 텐서플로 허브 모델을 사용하는 케라스 층을 만들어 보죠. 그다음 몇 개의 샘플을 입력하여 테스트해 보겠습니다. 입력 텍스트의 길이에 상관없이 임베딩의 출력 크기는 `(num_examples, embedding_dimension)`가 됩니다.

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

이제 전체 모델을 만들어 보겠습니다:

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


순서대로 층을 쌓아 분류기를 만듭니다:

1. 첫 번째 레이어는 TensorFlow Hub 레이어입니다. 이 레이어는 사전 훈련된 저장된 모델을 사용하여 문장을 임베딩 벡터에 매핑합니다. 사용 중인 사전 훈련된 텍스트 임베딩 모델([google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2))은 문장을 토큰으로 분할하고 각 토큰을 임베딩한 다음 임베딩을 결합합니다. 결과적인 차원은 `(num_examples, embedding_dimension)`입니다. 이 NNLM 모델의 경우에는 `embedding_dimension`은 50입니다.
2. 이 고정 크기의 출력 벡터는 16개의 은닉 유닛(hidden unit)을 가진 완전 연결 층(`Dense`)으로 주입됩니다.
3. 마지막 층은 하나의 출력 노드를 가진 완전 연결 층입니다. `sigmoid` 활성화 함수를 사용하므로 확률 또는 신뢰도 수준을 표현하는 0~1 사이의 실수가 출력됩니다.

이제 모델을 컴파일합니다.

### 손실 함수와 옵티마이저

모델에는 훈련을 위한 손실 함수와 옵티마이저가 필요합니다. 이진 분류 문제이고 모델이 로짓(선형 활성화가 있는 단일 단위 레이어)을 출력하므로 `binary_crossentropy` 손실 함수를 사용합니다.

다른 손실 함수를 선택할 수 없는 것은 아닙니다. 예를 들어 `mean_squared_error`를 선택할 수 있습니다. 하지만 일반적으로 `binary_crossentropy`가 확률을 다루는데 적합합니다. 이 함수는 확률 분포 간의 거리를 측정합니다. 여기에서는 정답인 타깃 분포와 예측 분포 사이의 거리입니다.

나중에 회귀 문제(예: 주택 가격 예측)를 살펴볼 때 평균 제곱 오차라고 하는 또 다른 손실 함수를 사용하는 방법을 살펴볼 것입니다.

이제 모델이 사용할 옵티마이저와 손실 함수를 설정해 보죠:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 모델 훈련

512개 샘플의 미니 배치에서 10개 epoch 동안 모델을 훈련합니다. 이 동작은 `x_train` 및 `y_train` 텐서의 모든 샘플에 대한 10회 반복에 해당합니다. 훈련하는 동안 검증 세트의 10,000개 샘플에서 모델의 손실과 정확도를 모니터링합니다.

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10


 1/30 [>.............................] - ETA: 39s - loss: 0.7994 - accuracy: 0.5176

 3/30 [==>...........................] - ETA: 0s - loss: 0.7980 - accuracy: 0.4922 

 5/30 [====>.........................] - ETA: 0s - loss: 0.7655 - accuracy: 0.4980

 7/30 [======>.......................] - ETA: 0s - loss: 0.7490 - accuracy: 0.5028

...

20/30 [===================>..........] - ETA: 0s - loss: 0.0502 - accuracy: 0.9914

23/30 [======================>.......] - ETA: 0s - loss: 0.0508 - accuracy: 0.9910

26/30 [=========================>....] - ETA: 0s - loss: 0.0496 - accuracy: 0.9915

29/30 [============================>.] - ETA: 0s - loss: 0.0490 - accuracy: 0.9914

30/30 [==============================] - 2s 46ms/step - loss: 0.0491 - accuracy: 0.9913 - val_loss: 0.3260 - val_accuracy: 0.8731


## 모델 평가

모델의 성능을 확인해 보죠. 두 개의 값이 반환됩니다. 손실(오차를 나타내는 숫자이므로 낮을수록 좋습니다)과 정확도입니다.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3497 - accuracy: 0.8561


loss: 0.350
accuracy: 0.856


이 예제는 매우 단순한 방식을 사용하므로 87% 정도의 정확도를 달성했습니다. 고급 방법을 사용한 모델은 95%에 가까운 정확도를 얻습니다.

## 더 읽을거리

- 문자열 입력으로 작업하는 보다 일반적인 방법과 훈련 중 정확도 및 손실의 진행 상황에 대한 보다 자세한 분석은 [전처리된 텍스트를 사용한 텍스트 분류](./text_classification.ipynb) 튜토리얼을 참조하세요.
- TFHub에서 훈련된 모델을 사용하여 더 많은 [텍스트 관련 튜토리얼](https://www.tensorflow.org/hub/tutorials#text-related-tutorials)을 시도해 보세요.